<a href="https://colab.research.google.com/github/Resia05/supervised-learning/blob/main/%D0%9A%D0%BE%D0%BF%D1%96%D1%8F_%D0%B7%D0%B0%D0%BF%D0%B8%D1%81%D0%BD%D0%B8%D0%BA%D0%B0_%22HW_2_9_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Імпорти

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

In [56]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

# Читання файлу, розбиття на колонки

In [66]:
# Читання даних з файлу
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/customer_segmentation_train.csv", index_col=0)
target_col = 'Segmentation'
X = df.drop(columns=target_col)
y = df[target_col]

# Розділимо дані на тренувальні та тестові набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Ідентифікація числових та категоріальних колонок
numeric_cols = X_train.select_dtypes(include=[np.number, 'int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()


In [50]:
# df.dtypes
# df.info()
# df.describe()
# закоментувала, бо вже ознайомилась з даними і на поточному етапі мені цей код заважає для подальшої роботи. Далі частина коду теж буде закоментована

# Робота з пропущеними значеннями

In [67]:
# Перевірка пропусків та нулів
def check_missing(df):
    # Кількість пропущених значень
    missing_count = df.isnull().sum()

    # Кількість нулів
    zero_count = (df == 0).sum()

    # Загальна кількість записів
    total_count = df.shape[0]

    # Відсоток пропущених значень
    missing_percentage = (missing_count / total_count) * 100

    # Відсоток нулів
    zero_percentage = (zero_count / total_count) * 100

    # Створення DataFrame для результатів
    result_df = pd.DataFrame({
        'Missing Count': missing_count,
        'Zero Count': zero_count,
        'Missing Percentage': missing_percentage,
        'Zero Percentage': zero_percentage
    })

    return result_df

# print("\nMissing and Zero Values in train_inputs:")
# missing_zero_train_inputs = check_missing(X_train)
# print(missing_zero_train_inputs)

# print("\nMissing and Zero Values in test_inputs:")
# missing_zero_test_inputs = check_missing(X_test)
# print(missing_zero_test_inputs)

In [68]:
def impute_missing_values(df):
    # Заміна пропущених значень в Ever_Married та Graduated на "Yes"
    df['Ever_Married'].fillna('Yes', inplace=True)
    df['Graduated'].fillna('Yes', inplace=True)

    # Заміна пропущених значень в Profession на "Other"
    df['Profession'].fillna('Other', inplace=True)

    # Заміна пропущених значень в Work_Experience на медіанне значення
    median_work_experience = df['Work_Experience'].median()
    df['Work_Experience'].fillna(median_work_experience, inplace=True)

    # Заміна пропущених значень в Family_Size на середнє значення
    mean_family_size = df['Family_Size'].mean()
    df['Family_Size'].fillna(mean_family_size, inplace=True)

    # Заміна пропущених значень в Var_1 на "Cat_6"
    df['Var_1'].fillna('Cat_6', inplace=True)

    return df

# Застосуємо функцію
X_train_imputed = impute_missing_values(X_train)
X_test_imputed = impute_missing_values(X_test)
df_imputed = impute_missing_values(df)

# Перевірка результатів для X_train
# print("Missing values in X_train:")
# print(X_train_imputed.isnull().sum())

# Перевірка результатів для X_test
# print("\nMissing values in X_test:")
# print(X_test_imputed.isnull().sum())

# Перевірка результатів для df
# print("\nMissing values in df:")
# print(df_imputed.isnull().sum())


# Препроцесінг (бінарна, скалери, кодування)

In [70]:
# Бінарна заміна для колонок 'Gender', 'Ever_Married', 'Graduated'
binary_cols = ['Gender', 'Ever_Married', 'Graduated']
X_train[binary_cols] = X_train[binary_cols].replace({'Male': 1, 'Female': 0, 'Yes': 1, 'No': 0})
X_test[binary_cols] = X_test[binary_cols].replace({'Male': 1, 'Female': 0, 'Yes': 1, 'No': 0})

# Визначення колонок для ванхотенкодінгу
categorical_cols = ['Profession', 'Spending_Score', 'Var_1']

# Закодування категорій у числові значення
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

# MinMax scaling (просто для прикладу)
scaler = MinMaxScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# One hot encoding для категоріальних колонок
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit(X_train[categorical_cols])

# Отримання закодованих ознак
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

# Трансформація даних з використанням OneHotEncoder
X_train_encoded = encoder.transform(X_train[categorical_cols])
X_test_encoded = encoder.transform(X_test[categorical_cols])

# Створення нового DataFrame з закодованими ознаками
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoded_cols, index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoded_cols, index=X_test.index)

# Додавання закодованих ознак до вихідних даних
X_train_processed = pd.concat([X_train[numeric_cols], X_train_encoded_df, X_train[binary_cols]], axis=1)
X_test_processed = pd.concat([X_test[numeric_cols], X_test_encoded_df, X_test[binary_cols]], axis=1)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Методи ресамплінгу

**Завдання 2**. Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках і порівняти лін регресію на апсампл даних без категоріальних ознак.

2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

In [72]:
X_train_processed

,Age,Work_Experience,Family_Size,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,...,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Gender,Ever_Married,Graduated
ID,,,,,,,,,,,,,,,,,,,,,
465905,0.197183,0.642857,0.000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1
462903,0.760563,0.071429,0.125,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,1
467901,0.211268,0.071429,0.375,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1
463613,0.422535,0.000000,0.625,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,1
459859,0.140845,0.642857,0.000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463101,0.126761,0.571429,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0
467844,0.267606,0.000000,0.125,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,1
460706,0.126761,0.428571,0.625,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0


In [73]:
y_train_encoded

array([0, 1, 1, ..., 3, 3, 0])

In [80]:
# Ініціалізація SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_processed, y_train_encoded)

# Ініціалізація SMOTE-Tomek
smtomek = SMOTETomek(random_state=42)
X_train_smtomek, y_train_smtomek = smtomek.fit_resample(X_train_processed, y_train_encoded)

# Розміри нових наборів даних після ресемплінгу
print("Розміри після SMOTE:", X_train_smote.shape, y_train_smote.shape)
print("Розміри після SMOTE-Tomek:", X_train_smtomek.shape, y_train_smtomek.shape)
print("Розмір оригінальної тренувальної вибірки:", X_train_processed.shape, y_train_encoded.shape)

Розміри після SMOTE: (7256, 26) (7256,)
Розміри після SMOTE-Tomek: (5694, 26) (5694,)
Розмір оригінальної тренувальної вибірки: (6454, 26) (6454,)


In [76]:
X_train_smote

,Age,Work_Experience,Family_Size,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,...,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Gender,Ever_Married,Graduated
0,0.197183,0.642857,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1
1,0.760563,0.071429,0.125000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,1
2,0.211268,0.071429,0.375000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1
3,0.422535,0.000000,0.625000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,1
4,0.140845,0.642857,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7251,0.758038,0.000000,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,1
7252,0.241962,0.480459,0.303066,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,1
7253,0.457655,0.071429,0.326479,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0
7254,0.323944,0.071429,0.366233,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,1


In [77]:
X_train_smtomek

,Age,Work_Experience,Family_Size,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,...,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Gender,Ever_Married,Graduated
0,0.197183,0.642857,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1
1,0.211268,0.071429,0.375000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1
2,0.140845,0.642857,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0
3,0.380282,0.214286,0.250000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,1
4,0.352113,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5689,0.942247,0.029587,0.125000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,1
5690,0.758038,0.000000,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,1
5691,0.241962,0.480459,0.303066,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,1
5692,0.457655,0.071429,0.326479,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,0


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

# one-vs-rest (OvR)

In [89]:
# Логістична регресія зі стратегією one-vs-rest (OvR)
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train_processed, y_train_encoded)
ovr_predictions = ovr_model.predict(X_test_processed)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_test_encoded, ovr_predictions))

              precision    recall  f1-score   support

           0       0.43      0.46      0.44       394
           1       0.42      0.17      0.24       372
           2       0.49      0.63      0.55       394
           3       0.65      0.76      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.48      1614
weighted avg       0.50      0.52      0.50      1614



In [93]:
# Логістична регресія зі стратегією one-vs-rest (OvR) на даних збалансованих SMOTE
log_reg_smote = LogisticRegression(solver='liblinear')
ovr_model_smote = OneVsRestClassifier(log_reg_smote)
ovr_model_smote.fit(X_train_smote, y_train_smote)
ovr_predictions_smote = ovr_model_smote.predict(X_test_processed)

# Обчислимо метрики precision та recall для кожного класу
print("Classification Report for Logistic Regression with SMOTE:")
print(classification_report(y_test_encoded, ovr_predictions_smote))


Classification Report for Logistic Regression with SMOTE:
              precision    recall  f1-score   support

           0       0.43      0.48      0.45       394
           1       0.40      0.25      0.31       372
           2       0.50      0.60      0.54       394
           3       0.68      0.71      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



In [94]:
# Логістична регресія зі стратегією one-vs-rest (OvR) на даних збалансованих SMOTE-Tomek
log_reg_smtomek = LogisticRegression(solver='liblinear')
ovr_model_smtomek = OneVsRestClassifier(log_reg_smtomek)
ovr_model_smtomek.fit(X_train_smtomek, y_train_smtomek)
ovr_predictions_smtomek = ovr_model_smtomek.predict(X_test_processed)

# Обчислимо метрики precision та recall для кожного класу
print("Classification Report for Logistic Regression with SMOTE-Tomek:")
print(classification_report(y_test_encoded, ovr_predictions_smtomek))


Classification Report for Logistic Regression with SMOTE-Tomek:
              precision    recall  f1-score   support

           0       0.43      0.48      0.46       394
           1       0.39      0.24      0.29       372
           2       0.49      0.60      0.54       394
           3       0.68      0.70      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



# one-vs-one (OvO)

In [91]:
# Логістична регресія зі стратегією one-vs-one (OvO)
ovo_model = OneVsOneClassifier(log_reg)
ovo_model.fit(X_train_processed, y_train_encoded)
ovo_predictions = ovo_model.predict(X_test_processed)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_test_encoded, ovo_predictions))


              precision    recall  f1-score   support

           0       0.43      0.41      0.42       394
           1       0.41      0.28      0.34       372
           2       0.52      0.59      0.55       394
           3       0.64      0.76      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



In [95]:
# Логістична регресія зі стратегією one-vs-one (OvO) на даних, збалансованих за допомогою SMOTE
log_reg = LogisticRegression(solver='liblinear')
ovo_model_smote = OneVsOneClassifier(log_reg)
ovo_model_smote.fit(X_train_smote, y_train_smote)
ovo_predictions_smote = ovo_model_smote.predict(X_test_processed)

# Обчислимо метрики precision та recall для кожного класу
print("Classification Report for Logistic Regression with OvO and SMOTE:")
print(classification_report(y_test_encoded, ovo_predictions_smote))

Classification Report for Logistic Regression with OvO and SMOTE:
              precision    recall  f1-score   support

           0       0.43      0.43      0.43       394
           1       0.41      0.33      0.37       372
           2       0.52      0.57      0.54       394
           3       0.66      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.51      1614
weighted avg       0.51      0.52      0.52      1614



In [96]:
# Логістична регресія зі стратегією one-vs-one (OvO) на даних, збалансованих за допомогою SMOTE-Tomek
log_reg = LogisticRegression(solver='liblinear')
ovo_model_smtomek = OneVsOneClassifier(log_reg)
ovo_model_smtomek.fit(X_train_smtomek, y_train_smtomek)
ovo_predictions_smtomek = ovo_model_smtomek.predict(X_test_processed)

# Обчислимо метрики precision та recall для кожного класу
print("Classification Report for Logistic Regression with OvO and SMOTE-Tomek:")
print(classification_report(y_test_encoded, ovo_predictions_smtomek))

Classification Report for Logistic Regression with OvO and SMOTE-Tomek:
              precision    recall  f1-score   support

           0       0.44      0.43      0.43       394
           1       0.40      0.34      0.37       372
           2       0.52      0.56      0.54       394
           3       0.66      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.51      1614
weighted avg       0.51      0.52      0.52      1614



# Усереднені метрики macro та micro

In [92]:
# Усереднені метрики macro та micro для OvR
ovr_macro_precision = precision_score(y_test_encoded, ovr_predictions, average='macro')
ovr_micro_precision = precision_score(y_test_encoded, ovr_predictions, average='micro')
ovr_macro_recall = recall_score(y_test_encoded, ovr_predictions, average='macro')
ovr_micro_recall = recall_score(y_test_encoded, ovr_predictions, average='micro')

# Усереднені метрики macro та micro для OvO
ovo_macro_precision = precision_score(y_test_encoded, ovo_predictions, average='macro')
ovo_micro_precision = precision_score(y_test_encoded, ovo_predictions, average='micro')
ovo_macro_recall = recall_score(y_test_encoded, ovo_predictions, average='macro')
ovo_micro_recall = recall_score(y_test_encoded, ovo_predictions, average='micro')

# Створимо датафрейм для відображення результатів
results = pd.DataFrame({
    'Metric': ['Macro Precision', 'Micro Precision', 'Macro Recall', 'Micro Recall'],
    'OvR': [ovr_macro_precision, ovr_micro_precision, ovr_macro_recall, ovr_micro_recall],
    'OvO': [ovo_macro_precision, ovo_micro_precision, ovo_macro_recall, ovo_micro_recall]
})
print(results)

            Metric       OvR       OvO
0  Macro Precision  0.496488  0.500703
1  Micro Precision  0.519207  0.521685
2     Macro Recall  0.505113  0.509441
3     Micro Recall  0.519207  0.521685


In [97]:
# Оцінки precision та recall для OvR з SMOTE
ovr_macro_precision_smote = precision_score(y_test_encoded, ovr_predictions_smote, average='macro')
ovr_micro_precision_smote = precision_score(y_test_encoded, ovr_predictions_smote, average='micro')
ovr_macro_recall_smote = recall_score(y_test_encoded, ovr_predictions_smote, average='macro')
ovr_micro_recall_smote = recall_score(y_test_encoded, ovr_predictions_smote, average='micro')

# Оцінки precision та recall для OvO з SMOTE
ovo_macro_precision_smote = precision_score(y_test_encoded, ovo_predictions_smote, average='macro')
ovo_micro_precision_smote = precision_score(y_test_encoded, ovo_predictions_smote, average='micro')
ovo_macro_recall_smote = recall_score(y_test_encoded, ovo_predictions_smote, average='macro')
ovo_micro_recall_smote = recall_score(y_test_encoded, ovo_predictions_smote, average='micro')

# Виведемо результати для SMOTE
print("Results for Logistic Regression with OvR and OvO using SMOTE:")
results_smote = pd.DataFrame({
    'Metric': ['Macro Precision', 'Micro Precision', 'Macro Recall', 'Micro Recall'],
    'OvR': [ovr_macro_precision_smote, ovr_micro_precision_smote, ovr_macro_recall_smote, ovr_micro_recall_smote],
    'OvO': [ovo_macro_precision_smote, ovo_micro_precision_smote, ovo_macro_recall_smote, ovo_micro_recall_smote]
})
print(results_smote)

Results for Logistic Regression with OvR and OvO using SMOTE:
            Metric       OvR       OvO
0  Macro Precision  0.502408  0.505765
1  Micro Precision  0.519827  0.521066
2     Macro Recall  0.508643  0.511029
3     Micro Recall  0.519827  0.521066


In [98]:
# Оцінки precision та recall для OvR з SMOTE-Tomek
ovr_macro_precision_smtomek = precision_score(y_test_encoded, ovr_predictions_smtomek, average='macro')
ovr_micro_precision_smtomek = precision_score(y_test_encoded, ovr_predictions_smtomek, average='micro')
ovr_macro_recall_smtomek = recall_score(y_test_encoded, ovr_predictions_smtomek, average='macro')
ovr_micro_recall_smtomek = recall_score(y_test_encoded, ovr_predictions_smtomek, average='micro')

# Оцінки precision та recall для OvO з SMOTE-Tomek
ovo_macro_precision_smtomek = precision_score(y_test_encoded, ovo_predictions_smtomek, average='macro')
ovo_micro_precision_smtomek = precision_score(y_test_encoded, ovo_predictions_smtomek, average='micro')
ovo_macro_recall_smtomek = recall_score(y_test_encoded, ovo_predictions_smtomek, average='macro')
ovo_micro_recall_smtomek = recall_score(y_test_encoded, ovo_predictions_smtomek, average='micro')

# Виведемо результати для SMOTE-Tomek
print("\nResults for Logistic Regression with OvR and OvO using SMOTE-Tomek:")
results_smtomek = pd.DataFrame({
    'Metric': ['Macro Precision', 'Micro Precision', 'Macro Recall', 'Micro Recall'],
    'OvR': [ovr_macro_precision_smtomek, ovr_micro_precision_smtomek, ovr_macro_recall_smtomek, ovr_micro_recall_smtomek],
    'OvO': [ovo_macro_precision_smtomek, ovo_micro_precision_smtomek, ovo_macro_recall_smtomek, ovo_micro_recall_smtomek]
})
print(results_smtomek)


Results for Logistic Regression with OvR and OvO using SMOTE-Tomek:
            Metric       OvR       OvO
0  Macro Precision  0.499351  0.506415
1  Micro Precision  0.516729  0.521685
2     Macro Recall  0.505740  0.511655
3     Micro Recall  0.516729  0.521685


In [100]:
# Створимо датафрейм для відображення результатів
results_comparison = pd.DataFrame({
    'Method': ['Original', 'SMOTE', 'SMOTE-Tomek'],
    'Macro Precision': [ovr_macro_precision, ovr_macro_precision_smote, ovr_macro_precision_smtomek],
    'Micro Precision': [ovr_micro_precision, ovr_micro_precision_smote, ovr_micro_precision_smtomek],
    'Macro Recall': [ovr_macro_recall, ovr_macro_recall_smote, ovr_macro_recall_smtomek],
    'Micro Recall': [ovr_micro_recall, ovr_micro_recall_smote, ovr_micro_recall_smtomek]
})

print("Comparison of Logistic Regression with One-vs-Rest Strategy (OvR) on Different Data Balancing Techniques:")
print(results_comparison)

Comparison of Logistic Regression with One-vs-Rest Strategy (OvR) on Different Data Balancing Techniques:
        Method  Macro Precision  Micro Precision  Macro Recall  Micro Recall
0     Original         0.496488         0.519207      0.505113      0.519207
1        SMOTE         0.502408         0.519827      0.508643      0.519827
2  SMOTE-Tomek         0.499351         0.516729      0.505740      0.516729


Результати суттєво не відрізняються. Дещо кращим є SMOTE.
Ймовірно, що це через хорошу підготовку даних (заміна пропусків, нормалізація, кодування і тд).

In [104]:
# Визначимо мету та функції
target_col = 'Segmentation'
X = df_imputed.drop(columns=target_col)
y = df_imputed[target_col]

# Розділимо дані на тренувальні та тестові набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Ідентифікація числових та категоріальних колонок
numeric_cols = X_train.select_dtypes(include=[np.number, 'int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()

# Виберемо лише числові дані для моделі
X_train_numeric = X_train[numeric_cols]
X_test_numeric = X_test[numeric_cols]

# Ініціалізація кодувальника міток і його навчання на тренувальних даних
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Ініціалізуємо модель логістичної регресії зі стратегією OvR
log_reg = LogisticRegression(solver='liblinear', max_iter=1000)
ovr_model_numeric = OneVsRestClassifier(log_reg)

# Натренуємо модель на числових даних
ovr_model_numeric.fit(X_train_numeric, y_train_encoded)

# Зробимо прогнози
ovr_predictions_numeric = ovr_model_numeric.predict(X_test_numeric)

# Обчислимо метрики precision та recall для числових даних
macro_precision_numeric = precision_score(y_test_encoded, ovr_predictions_numeric, average='macro')
micro_precision_numeric = precision_score(y_test_encoded, ovr_predictions_numeric, average='micro')
macro_recall_numeric = recall_score(y_test_encoded, ovr_predictions_numeric, average='macro')
micro_recall_numeric = recall_score(y_test_encoded, ovr_predictions_numeric, average='micro')

# Виведемо результати
print("Classification Report for Logistic Regression with One-vs-Rest Strategy (OvR) on Numeric Data:")
print(classification_report(y_test_encoded, ovr_predictions_numeric))

# Виведемо усереднені метрики
results_numeric = pd.DataFrame({
    'Metric': ['Macro Precision', 'Micro Precision', 'Macro Recall', 'Micro Recall'],
    'Numeric Data': [macro_precision_numeric, micro_precision_numeric, macro_recall_numeric, micro_recall_numeric]
})
print("\nAverage Metrics for Logistic Regression with One-vs-Rest Strategy (OvR) on Numeric Data:")
print(results_numeric)

Classification Report for Logistic Regression with One-vs-Rest Strategy (OvR) on Numeric Data:
              precision    recall  f1-score   support

           0       0.33      0.23      0.27       394
           1       0.45      0.02      0.05       372
           2       0.37      0.48      0.42       394
           3       0.45      0.80      0.58       454

    accuracy                           0.41      1614
   macro avg       0.40      0.39      0.33      1614
weighted avg       0.40      0.41      0.34      1614


Average Metrics for Logistic Regression with One-vs-Rest Strategy (OvR) on Numeric Data:
            Metric  Numeric Data
0  Macro Precision      0.401234
1  Micro Precision      0.406444
2     Macro Recall      0.385974
3     Micro Recall      0.406444


Тепер дійсно видно різницю в результатах, коли дані не підготовлені. Попередні розрахунки були кращими.